In [1]:
import os
import random
import pygame
import mediapipe as mp
import cv2
import numpy as np
from Fruit import Fruit
from HandTrackingModule import HandDetector  # Assuming you have HandTrackingModule in the same directory
import pymunk
import time


pygame 2.5.2 (SDL 2.28.3, Python 3.7.1)
Hello from the pygame community. https://www.pygame.org/contribute.html


### 전역변수 선언


In [2]:
# Pygame 초기화
pygame.init() #Pygame 라이브러리 초기화.
pygame.mixer.init()  # 소리 초기화를 위한 라인 추가
pygame.event.clear() #이벤트 초기화.

# 창/화면 생성
width, height = 1300, 700
window = pygame.display.set_mode((1200, 700)) #창의 크기를 1200x700으로 설정.
pygame.display.set_caption("Fruit Slicer") #창의 제목 설정.

# FPS용 시계 초기화
fps = 23 #초당 프레임 수 설정.
clock = pygame.time.Clock() #FPS 제어용 clock 객체 생성.

# 게임 오버 이미지를 리사이징하여 로드
imgGameOver = pygame.transform.scale(pygame.image.load(r"C:\Users\bluecom011\Desktop\Fruit/fru.jpg").convert(), (1300, 800))

# Mediapipe 포즈 클래스 초기화
mp_pose = mp.solutions.pose
pose = mp_pose.Pose()

# 웹캠 설정
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)  # 너비
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)  # 높이

# Hand detector 초기화
detector = HandDetector(maxHands=1, detectionCon=0.8)
# 손을 감지하는데 사용될 Hand Detector를 초기화합니다. 최대 1개의 손을 감지하며, 감지 신뢰도는 0.8로 설정합니다.

# Pymunk을 사용하여 물리 공간을 초기화합니다. 중력을 (0.0, -1000.0)으로 설정합니다.
space = pymunk.Space()
space.gravity = 0.0, -1000.0

# 게임 변수 및 파라미터 초기화
timeTotal = 10  # 게임 총 시간(초)
comboTimeLimit = 1  # Combo 시간 제한(초)
fruitList = []  # 생성된 과일 객체를 저장할 리스트
timeGenerator = time.time() #현재 시간을 저장하여 게임에 사용될 시간을 생성하는 데 활용
timeStart = time.time() #게임 시작 시간을 저장
gameOver = False #게임 종료 여부를 나타내는 변수
totalScore = 0 #게임의 총 점수
blue = (255, 127, 0)
COLOR = (204, 204, 255)      
comboCount = 0 #Combo 횟수를 나타내는 변수
lastSliceTime = time.time() #마지막으로 과일을 베어낸 시간을 저장

# 과일 경로 목록
pathFruitFolder = r"C:\Users\bluecom011\Desktop\Fruit/Fruits"
## 과일 이미지 파일이 저장된 폴더의 경로를 지정하고, 해당 폴더 안의 모든 파일명을 리스트로 저장합니다.
pathListFruit = os.listdir(pathFruitFolder)

# 전체 점수를 누적할 변수
totalScore = 0

## 게임 초기화 함수

In [3]:
# 게임 초기화 함수
def reset_game():
    # 함수 내에서 전역 변수(global)를 수정하기 위해 global 키워드를 사용합니다.
    global totalScore, timeGenerator, timeStart, currentRound, gameOver, fruitList, comboCount
    # 게임 총 점수를 0으로 초기화합니다.
    totalScore = 0
    # 현재 시간을 새로운 값으로 갱신하여 게임 시작 시간을 초기화합니다.
    timeGenerator = time.time()
    timeStart = time.time()
    # 현재 라운드를 1로 초기화합니다.
    currentRound = 1
    # 게임 종료 여부를 나타내는 변수를 False로 초기화합니다.
    gameOver = False
    # 생성된 과일 객체를 저장하는 리스트를 비웁니다.
    fruitList = []
    # Combo 횟수를 0으로 초기화합니다. (콤보 초기화 추가)
    comboCount = 0

### 과일생성 함수

In [4]:
# 과일 생성 함수
def generate_fruit():
    # 랜덤한 크기 및 과일 이미지 경로 선택
    # 0.6에서 0.8 사이의 랜덤한 크기(scale)를 선택합니다.
    randomScale = round(random.uniform(0.6, 0.8), 2)
    # 과일 이미지 파일들의 경로 리스트에서 랜덤하게 하나를 선택합니다.
    randomFruitPath = pathListFruit[random.randint(0, len(pathListFruit) - 1)]

    # 폭탄인 경우와 그렇지 않은 경우에 따라 소리 경로 선택
    # 과일 이미지 파일 경로에 "bomb"이 포함되어 있는지 확인하여, 해당 여부에 따라 소리 경로를 선택합니다.
    # "bomb"이 포함되어 있으면 폭발 소리 경로, 그렇지 않으면 일반 과일 베는 소리 경로를 선택합니다.
    pathSoundSlice = r'C:\Users\bluecom011\Desktop\Fruit/explosion.wav' if "bomb" in randomFruitPath else r'C:\Users\bluecom011\Desktop\Fruit/slice.wav'
    # Fruit 클래스의 객체를 생성하여 fruitList에 추가
    fruitList.append(Fruit(space, path=os.path.join(pathFruitFolder, randomFruitPath),
                           grid=(4, 4), animationFrames=14, scale=randomScale,
                           pathSoundSlice=pathSoundSlice))
# - space: Pymunk 물리 공간
    # - path: 과일 이미지 파일의 경로
    # - grid: 과일 이미지 파일을 나눌 격자의 행과 열 수
    # - animationFrames: 과일 이미지 애니메이션 프레임 수
    # - scale: 과일의 크기(scale)
    # - pathSoundSlice: 과일을 베었을 때 재생할 소리 파일 경로

# 리스타트 버튼 함수

### 게임 재시작 버튼 함수

In [5]:
def restart_button():
    global gameOver, totalScore, timeGenerator, timeStart, fruitList, currentRound, comboCount
    # 버튼 크기 및 위치 설정
    button_width, button_height = 200, 50
    button_x, button_y = (width - button_width) // 2, height // 2 + 100
    # 버튼을 그립니다.
    pygame.draw.rect(window, (0, 255, 0), (button_x, button_y, button_width, button_height))
    # 마우스의 위치 및 클릭 여부 확인
    mouse_x, mouse_y = pygame.mouse.get_pos()
    click, _, _ = pygame.mouse.get_pressed()
    # 마우스가 버튼 위에 있는지 확인하고 클릭하면 게임을 다시 시작합니다.
    if button_x < mouse_x < button_x + button_width and button_y < mouse_y < button_y + button_height:
        pygame.draw.rect(window, (0, 200, 0), (button_x, button_y, button_width, button_height))
        if click:
            # 게임 변수 초기화
            reset_game()
            # 추가적인 초기화 로직을 여기에 추가할 수 있습니다.
            # 예를 들어, 배경 음악을 다시 로드하고 재생할 수 있습니다.
            pygame.mixer.music.load(r"C:\Users\bluecom011\Desktop\Fruit/background.mp3")
            pygame.mixer.music.play(-1)
    # 버튼에 텍스트 추가
    font = pygame.font.Font(None, 36)
    restart_text = font.render("Restart", True, (255, 255, 255))
    text_rect = restart_text.get_rect(center=(button_x + button_width // 2, button_y + button_height // 2))
    window.blit(restart_text, text_rect)

In [6]:
# 게임 루프 함수
def game_loop():
    global gameOver, totalScore, timeGenerator, timeStart, currentRound, comboCount, lastSliceTime
    # 현재 라운드 및 총 라운드 설정
    currentRound = 1
    totalRounds = 100
    comboCount = 0 
    # 배경 음악 로드 및 재생
    pygame.mixer.music.load(r"C:\Users\bluecom011\Desktop\Fruit/background.mp3")
    pygame.mixer.music.play(-1)

    while cap.isOpened() and currentRound <= totalRounds:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                return
        # 웹캠에서 프레임 읽기
        if gameOver is False:
            success, img = cap.read()
            img = cv2.flip(img, 1)
            h, w = img.shape[:2]
            # 이미지 컬러 변환
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # Mediapipe를 사용하여 포즈 검출
            keypoints = pose.process(img)
             # 이미지를 원래대로 변환
            img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
            # 포즈 랜드마크 가져오기
            lm = keypoints.pose_landmarks
            lmPose = mp_pose.PoseLandmark

            try:
                # 오른손 검지 손가락 끝의 x, y 좌표 계산
                index_finger_tip_x = int(lm.landmark[lmPose.RIGHT_INDEX].x * w)
                index_finger_tip_y = int(lm.landmark[lmPose.RIGHT_INDEX].y * h)
                # 손 검출
                hands, img = detector.findHands(img)

                if hands:
                    # 손이 검출되면 손가락 끝의 좌표 업데이트
                    lmlist = hands[0]['lmList']
                    ind_x, ind_y = lmlist[8][0], lmlist[8][1]
                    index_finger_tip_x, index_finger_tip_y = ind_x, ind_y
                # 손가락 끝에 파란색 동그라미 그리기
                cv2.circle(img, (index_finger_tip_x, index_finger_tip_y), 20, COLOR, -1)
            except AttributeError:
                # 손가락이 감지되지 않으면 계속 진행
                continue
            # OpenCV 이미지를 Pygame Surface로 변환
            imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            # 이미지를 90도 회전
            imgRGB = np.rot90(imgRGB)
            # Pygame Surface로 변환
            frame = pygame.surfarray.make_surface(imgRGB).convert()
            # 이미지 좌우 반전(거울모드)
            frame = pygame.transform.flip(frame, True, False)
            # 프레임을 창에 표시
            window.blit(frame, (0, 0))
            # 과일 생성 및 손가락 위치 업데이트
            if time.time() - timeGenerator > 1:
                # generate_fruit 함수로 과일을 생성
                generate_fruit()
                timeGenerator = time.time()

            x, y = index_finger_tip_x, index_finger_tip_y
            # Combo 여부에 따라 점수 배수 설정
            comboMultiplier = 2 if comboCount > 0 else 1
            # 생성된 과일들에 대해 처리
            comboText = None
            for i, fruit in enumerate(fruitList):
                if fruit:
                    # 과일을 화면에 그림
                    fruit.draw(window)
                    # 손가락 위치와 과일 충돌 체크
                    checkSlice = fruit.checkSlice(x, y)

                    if checkSlice == 2:
                        # 게임 종료
                        gameOver = True
                        # 배경 음악 정지
                        pygame.mixer.music.stop()
                    # 과일을 슬라이스한 경우 과일 리스트에서 제거
                    if checkSlice == 1:
                        fruitList[i] = False

                        # 현재 시간 기록
                        currentTime = time.time()

                        # 일정 시간 안에 슬라이스가 발생하면 Combo!
                        if currentTime - lastSliceTime <= comboTimeLimit:
                            comboCount += 1
                            # Combo 텍스트 업데이트
                            comboText = f"Combo x{comboCount}"

                            # Combo 중일 때만 점수 계산 로직 업데이트
                            totalScore += comboMultiplier  # Combo 배수에 따라 점수 조정

                        else:
                            # Combo가 아니면 점수를 1 증가
                            totalScore += 1

                        # 마지막 슬라이스 시간 갱신
                        lastSliceTime = currentTime
            # 남은 시간 계산
            timeLeft = int(timeTotal - (time.time() - timeStart))

            # 시간이 다 된 경우 다음 라운드 시작 또는 게임 종료
            if timeLeft <= 0:
                if currentRound < totalRounds:
                    # 다음 라운드로 진행
                    currentRound += 1
                    # 현재 라운드 시작 시간 초기화
                    timeStart = time.time()
                    pygame.mixer.music.load(r"C:\Users\bluecom011\Desktop\Fruit/background.mp3")
                    pygame.mixer.music.play(-1) # 배경 음악 재생

                    # 다음 라운드 시작 메시지 출력
                    font = pygame.font.Font(None, 60)
                    round_start_text = font.render(f"{currentRound}Round Start!", True, (255,0, 0))
                    text_rect = round_start_text.get_rect(center=(width // 2, height // 2))
                    window.blit(round_start_text, text_rect)
                    pygame.display.update()
                    pygame.time.delay(500)  # 0.5초 동안 메시지 표시

                else:
                    # 게임 종료
                    gameOver = True
                    # 배경 음악 정지
                    pygame.mixer.music.stop()
            # Combo 텍스트 표시
            if comboText:
                font = pygame.font.Font(None, 60)
                textCombo = font.render(comboText, True, (255, 255, 0))
                window.blit(textCombo, (width // 2 - 80, 100))

            # 점수 및 남은 시간 표시
            font = pygame.font.Font(None, 60)
            textScore = font.render(str(totalScore), True, blue)
            textTime = font.render(str(timeLeft), True, blue)
            window.blit(textScore, (225, 35))
            window.blit(textTime, (1000, 37))

        else:
            # 게임 오버 화면 표시
            window.blit(imgGameOver, (0, 0))

            # "You Lose!" 텍스트 및 점수, 라운드 표시
            font = pygame.font.Font(None, 150)
            textLose = font.render("You Lose!", True, (255, 0, 255) )
            textYour = font.render("Your Score:", True, (255, 0, 255) )
            textyou = font.render("Your Round:", True, (255, 0, 255) )
            textScore = font.render(str(totalScore), True, (255, 0, 255) )
            textRound = font.render(str(currentRound), True, (255, 0, 255) )
            window.blit(textLose, (400, 43))
            window.blit(textYour, (350, 143))
            window.blit(textScore, (950, 143))
            window.blit(textRound, (1040, 243))
            window.blit(textyou, (350, 243))

            # 재시작 버튼 표시
            restart_button()
        # 화면 업데이트
        pygame.display.update()
        clock.tick(fps)
        space.step(1 / fps)
        #1초 동안의 물리 시뮬레이션 단계를 나타냄, 프레임 속도에 맞춰 물리 시뮬레이션을 조절

    # Pygame 종료
    pygame.quit()

if __name__ == "__main__":
    game_loop()
